# Assignment 3 - Recommender Systems



The main idea to this assignment is learning and applying the concept of recommender systems. Imagine you are part of a company selling video games. You have users and you want to recommend games for these users to buy and play.

In [ ]:
# First some imports, as always
import torch
import pandas as pd
import numpy as np
from sklearn.metrics import pairwise_distances
from sklearn.decomposition import TruncatedSVD

In [ ]:
# Here we import two datasets, that we will later combine into one

steam_user_data = pd.read_csv('steam-200k.csv')
steam_description_data = pd.read_csv('steam_games.csv')

Ok let's take a look at our **steam_user_data** dataset. Each row contains a game (**game**) that was bought and/or played (**action**) by a user (**userid**). For a game that was played we addtionally know how long the user played the game (**hours**). Generally for each entry corresponding to a bought game, we also have an entry with the *play* action, but a user could buy a game, but never play it. Then we have an entry with the *purchase* action, but not an entry with a *play*. The hour field for a *purchase* entry is always 1. Column **other** can be ignored.

In [ ]:
steam_user_data.head(5)

We additionally have the **steam_description_data** set, which contains addtional information about each game. We will only need a few features from this. For example the **genre** of the game or the user-generated **popular_tags**.

In [ ]:
small_steam_description_data = steam_description_data[['game', 'genre', 'game_details', 'popular_tags', 'game_description']]
small_steam_description_data.head(3)

We join both datasets on the **game** attribute using pandas merge function.

In [ ]:
steam_data = steam_user_data.merge(small_steam_description_data, on='game')
steam_data.head(10)

## 1. Trivial Recommendations

So now we want to use this dataset to find out which one a user should play next. The first idea you might have is to just recommend the most popular titles. Now try to find the 10 most bought games according to our dataset. 

*Hint: Use the pandas groupby function*

In [ ]:
purchased = steam_data[steam_data['action'] == 'purchase']  # TODO (1 Point)
sorted_purchased = purchased.groupby('game')['game'].count().sort_values(ascending=False).reset_index(name='count').head(10) # TODO (1 Point)
sorted_purchased

Now these might be the most bought games, but not the most played games. There might be games that users buy, but not really play. Also games with a high avg playing time might mean, that users tend to like the game. Now try to find the 10 games with the longest avg playtime per user.

*Hint: Don't forget to select only the entries that contain played games*

In [ ]:
played = steam_data[steam_data['action'] == 'play']   # TODO (1 Point)
sorted_by_playtime = played.groupby('game',as_index=False).hours.mean().sort_values(by = 'hours', ascending = False).head(10) # TODO (1 Point)
sorted_by_playtime

This might actually have a slight problem. For example, WTF is 'Eastside Hockey Manager' and why does it have an average playing time of over 1000 hours. In order to migitage that some games might only be bought by very few users, that then play it a long time, let's weight the avg playtime by the percentage of users that play this game.

In [ ]:
# create ratio column (1 Point)
total_players_count = played['userid'].count()
players_per_game = played.groupby('game')['userid'].count().sort_values(ascending=False).reset_index(name='count')
df_ratio = players_per_game['count']/total_players_count
df_played = pd.DataFrame(played) 

# join with average playtimes from above, multiplication with average hours and get top 10 (3 Points)
df_tmp = df_played.join(df_ratio).rename(columns={'hours': 'avg_hours'})
df_tmp['playtime_user_ratio'] = df_ratio * df_tmp['avg_hours']# TODO (1 Point)
sorted_by_playtime_user_ratio = df_tmp.sort_values('playtime_user_ratio', ascending=False)# TODO (1 Point)
# sorted_by_playtime_user_ratio... # TODO (1 Point)

Now that should look like a pretty decent list of popular games on Steam from 4 years ago, when this dataset was collected. These lists might seem like a good starting point for a new user, but for old users we want to recommend games tat fit a user's preferences. So the first thing we can do is to look at the obvious features of the games that a user likes, e.g. genre, tags etc. and try to find similiar games with similiar features.

In [ ]:
sorted_by_playtime_user_ratio

## 2. Content-Based Recommendation

So let us imagine we have a user that really loves the rougelite rythm dungeon crawler *Crypt of the NecroDancer* (https://store.steampowered.com/app/247080/Crypt_of_the_NecroDancer/) Your task is now (in increasingly complicated and mathy ways) to find similiar games to it. We will begin by using the Jaccard Similarity measure and the **popular_tags** feature. The user generated tags should tell us a lot about what the content of the games are and just by comparing which games have simliar sets of tags should already give us a good collection of simliar games.

In [ ]:
tag_data = steam_data.groupby('game').popular_tags.first().reset_index(name = 'popular_tags') # TODO (1 Point)
tag_data.set_index('game', inplace=True)
tag_data.head(5)

*Encode the genre feature with One-Hot-Encoding by using panda's str.get_dummies(seperator) (https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.str.get_dummies.html) function*


In [ ]:
tag_dummies = tag_data['popular_tags'].str.get_dummies(sep=',') # TODO (1 Point)
# tag_dummies.head(5)

In [ ]:
tag_dummies.head()

*Hint: When dealing with binary data the jaccard distance (1 - jaccard simliarity) is actually equivalent to the hamming distance. Check the trivial example below and get familiar with the method pairwise_distances and hamming distance first. How would you calculate by hand?* 

In [ ]:
matrix = np.array([
    [0,1,1,0],
    [0,1,1,1],
    [0,0,0,0]]
)

row_similarity_distances =  pairwise_distances(matrix, metric = "hamming")# pairw... #TODO 
row_similarity_distances

In [ ]:
# Calculate the pairwise jaccard distances between each game using our popular tags dummies 
jac_dist =  pairwise_distances(tag_dummies, metric = "hamming")# TODO (1 Point)
jac_dist

*Hint: Try using numpy's argsort function and don't forget to not count the queried game itself*

In [ ]:
def getTopKSimiliarGames(distances: np.array, games: list, game: str, k: int):
    
    anchor_game_idx = games.index(game)

    #TODO START:
    anchor_game_array = distances[anchor_game_idx]
    sorting_array = np.argsort(anchor_game_array)
    similar_games_indecis = sorting_array[1:k+1]
    top_k_similar_games =  np.array(games)[similar_games_indecis].tolist()
    
    
    #TODO END
    
    return top_k_similar_games

Now let's use your function to find the top-10 most simliar games to 'Crypt of the NecroDancer'

In [ ]:
getTopKSimiliarGames(jac_dist, list(tag_data.index), 'Crypt of the NecroDancer', 10)

Now that already looks like a decent list of recommendations for someone who likes Dungeon Crawlers. But if we look at a user we actually know more than just one game they like. We need a way to combine the features of multiple games into a single query, so that we find the games that are most simliar to the ones the user likes.

We can combine the recommendations for several games (the ones a user played) by using TF/IDF and Cosine Distance. TF is the term frequency, which in this case means how often a tag is included in the games that a single user plays and it is compared with the IDF. IDF is the Indirect Document Frequency and in our case this is the count of all tag over all documents divided by the occurance of a single tag in all documents.

*TODO: Use the tag_dummies table to calculate the inverse Document Frequency for each tag using the follwing function, with N being the number of games and n_t being the occurence of a the queried tag overall documents:*
$$ \log_{10}\frac{N}{n_t}$$

In [ ]:
import math

def get_idf(term, terms, documents, occurancies):
    
    # TODO START (3 Points)
    index_term = terms.index(term)
    n_t = occurancies[:,index_term].sum()
    N = len(documents)
    term_idf = math.log10( N/n_t )
    
    
    
    # TODO END
    
    return term_idf

    
games = list(set(list(tag_dummies.index)))
popular_tags = list(tag_dummies.columns)

get_idf(term='2.5D', terms=popular_tags, documents=games, occurancies=tag_dummies.values)

In [ ]:
tag_idfs = [get_idf(term, popular_tags, games, tag_dummies.values) for term in popular_tags]
tag_idfs


*TODO: Replace all the 1 entries in the tag_dummies table with the corresponding idf value, resulting in a new vector space for our games*

*Hint: Try multiplying each column with the corresponding value by multiplying each row vector with tag_idfs vector - but without the usage of df.values or numpy in order to avoid floating point errors* 

In [ ]:
tag_idf_embedding = pd.DataFrame()

tag_idf_embedding = tag_dummies*tag_idfs # TODO (1 Point)
tag_idf_embedding

Then let's calculate the pairwise cosine distances and apply our topk function again.

In [ ]:
idf_distances = pairwise_distances(tag_idf_embedding, metric = "cosine")

In [ ]:
getTopKSimiliarGames(idf_distances, list(tag_data.index), 'Crypt of the NecroDancer', 10)

Interestingly enough, now you might see that a few Rythm Style games have found their way to the top. NecroDancer is a rougelike dungeon crawler and a rythm game. So  there are less rythm games than dungeon crawlers on steam and therefore the tags that concern the rythm part of the game are weighted more than the dungeon tags. And that is excactly the idea behind idf! We want to lessen the importance of more frequently used words, e.g. in language this would mean that words like 'the' and 'and' are explicitly embedded as less important than very words that are specific to a document. 

## 3. Collaborative Filtering

What we did up till now is often refered to as *Content-Based Recommendation*. This means we use the features of the items that a user likes or has interacted with (in this case video games) to find similiar items that this user might like. That means we do not use information about other user than the target. What we can also do is use the preferences of other users to recommend games to our target user.

And that is what *Collaborative Filtering* Recommender Systems try to do. In our case the information about games based upon other users is encoded in the **action** feature. We can look at this feature as a sort of user rating of a game. We assume that if someone bought a game they found it interesting enough and if they then played it a lot they they even liked the game. From this we want to create a matrix. 

*Hint: Use the pandas function pivot_table (https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.pivot_table.html)*

In [ ]:
purchased = steam_data.copy()
purchased['purchased'] =  purchased['action'].apply(lambda x: 1 if x == 'purchase' else 0)# TODO  (1 Point)
ratings_table = pd.pivot_table(purchased, values='purchased', index=['userid', 'game'],
                    columns=['action'], aggfunc=np.sum, fill_value=0) # TODO  (1 Point)
ratings_table

Another possibility would be to user the played hours as a rating, for example by fitting a histogram with 5 buckets (or even a mixture of gaussians with 5 functions) to the **hours** feature and using this as a rating vom 1 - 5 'stars'. This is also completely reasonable and if you want you can try it out, but we will continue with the purchase information + confidence model.

Let us take a closer look at our ratings matrix. What attributes can we gather from it?

*TODO: Calculate the percentage of 1 entries in the matrix*

In [ ]:
# sparsity =pd.DataFrame(ratings_table['purchase'].tolist()).isin([1]).sum(axis=1) # TODO (1 Point)
# print('{:.2%} of the user-item matrix is filled'.format(sparsity))

In [ ]:
s = pd.DataFrame(ratings_table['purchase'].tolist()).isin([1]).sum(axis=1)
# s

In [ ]:
steam_data[steam_data['action'] == 'purchase'].describe()

As you can see most of our table is empty. This means our matrix is very sparse! Which makes sense since this is in the title of your Assigment ;).

### 3.1 Trivial Calculations

So far our matrix had a row for each users showing us, but if we now transpose our matrix, we can have new features (users) for each of our games. 

In [ ]:
rating_data = pd.DataFrame(ratings_table['play'])
rating_data.index[0][1]

In [ ]:
# From now on we will use numpy instead of pandas, because some calculations 
# are easier done with numpy arrays instead of DataFrames
# Here is how to convert a pandas df to a numpy array
ratings_matrix = ratings_table.values
# Well that was easy:
ratings_matrix

In [ ]:
ratings_matrix.T

We can use now the resulting feature space to find simliar games again:

In [ ]:
# use sklearn pairwise distance to calculte the cosine distances 
trivial_cos_distances = pairwise_distances(ratings_matrix, metric='hamming')

In [ ]:
trivial_cos_distances

Now again let's Use your getTopKSimiliarGames function to find the 10 most similiar games to Crypt.

In [ ]:
getTopKSimiliarGames(trivial_cos_distances, list([x[1] for x in rating_data.index]), 'Crypt of the NecroDancer', 10)

In [ ]:
# Well that's actually not the worst result. Let's look at the distribution  
# our distances

game_index = list([x[1] for x in rating_data.index]).index('Crypt of the NecroDancer')
distances_to_game = trivial_cos_distances[game_index]

pd.DataFrame(distances_to_game).hist()

As you can see these distances are not distributed well. Which is excactly the problem with such sparse vector spaces. And if you listened to the lecture you might know what this calls for. Dimensionality Reduction with ...

### 3.2 SVD

So we can assume that there is some underlying factors that lead to users buying certain games. We do not know these attributes and therefore they are often refered to as *hidden features*, *latent features* or *embeddings*. We can assume that this latent space has a significantly lower dimensionality than our ratings_matrix. One way to find these features is to use the Matrix Factorization to create two low dimensional matrices that multiply to our ratings matrix. This can be seen as a kind of compression. One classic MF is SVD. In this case we want to reduce the dimensionality so we use truncated svd.

*Hint: Use sklearn.decomposition.TruncatedSVD (https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.TruncatedSVD.html) to create a new lower rank matrix. Start by using a high number of components (ca. 100) and look at the explained variance ratio. Use this information to find a reasonable number of dimensions after which the variance ratio drops below 0.01. The TruncatedSVD Functions is sometimes a bit unstable. If you get a weird error, just run the cell again until it runs through*

In [ ]:
from sklearn.decomposition import TruncatedSVD


metrics = {
    'comps': [],
    'avg_var_ratio': [],
    'usv': []
}
avg = 0
steps = 1
components = 100
svd = TruncatedSVD(components, n_iter=1)
#TODO START
while components > 0:
   
    if components % 5 == 0:
        svd.fit(ratings_matrix.T)
        avg = svd.explained_variance_ratio_.mean()
        metrics['comps'].append(components)
        metrics['avg_var_ratio'].append(avg)
        metrics['usv'].append(np.diag(svd.singular_values_))
        print(f"comps={components} avg_var_ratio={avg}")
    components-=steps
#TODO END

sufficient_num_comps = int(pd.DataFrame(metrics).tail().iloc[-2:-1,0])
print(f"sufficient_num_comps={sufficient_num_comps}") 

usv = pd.DataFrame(metrics).tail().iloc[-2:-1,2].ravel()[0]

*Use pairwise cosine distance on the reduced game vectors*

In [ ]:
svd_cos_distances = pairwise_distances(usv, metric='cosine')

In [ ]:
svd_cos_distances

And we run our function to search similiar games to 'Crypt of the Necrodancer' again

In [ ]:
getTopKSimiliarGames(svd_cos_distances, list([x[1] for x in rating_data.index]), 'Alien Swarm', 10)


Ok now you yourself can decide whether those are some good recommendations or not :P And that's it for this Assignment. I hope it was interesting and fun :)